In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pandas as pd
from scipy.optimize import fsolve 
import sys
import time
from scipy import optimize
from numpy import savetxt

In [2]:
def intercept(s, N):
    r = np.sqrt(s/(np.tan(np.pi/N)*N))
    return r

In [3]:
N = 6
area = 1
theta_ = (2*np.pi)/N
r = intercept(area, N)
c_max = np.power(-1, N)*np.power(-r,N)
#####
a1 = np.sin(theta_)
b1 = np.cos(theta_)
a2 = np.sin(2*theta_)
b2 = np.cos(2*theta_)
a3 = np.sin(3*theta_)
b3 = np.cos(3*theta_)
a4 = np.sin(4*theta_)
b4 = np.cos(4*theta_)
a5 = np.sin(5*theta_)
b5 = np.cos(5*theta_)
a6 = np.sin(6*theta_)
b6 = np.cos(6*theta_)
#####

In [4]:
def r2fun(R, *data):#polar coordinate
    '''this function describes the shape of droplet on a hydrophobic
    '''
    global theta_, r, N, c_max
    theta, c, m= data
    f=1
    x = R*np.cos(theta)
    y = R*np.sin(theta)
    factor = (1 + (c/c_max))**m
    for echo in range(N):
        i = echo+1
        fi = ((np.sin(i*theta_))*(y/factor) + np.cos(i*theta_)*(x/factor))-r
        f = f*fi
    f = np.power(-1, N)*f
    return f-c

In [5]:
def r2fun_initial(R, *data):#polar coordinate
    '''this function describes the shape of droplet on a hydrophobic
    '''
    global theta_, r, N, c_max
    theta, c, m= data
    f=1
    x = R*np.cos(theta)
    y = R*np.sin(theta)
    factor = (1 + (c/c_max))**m
    for echo in range(N):
        i = echo+1
        fi = ((np.sin(i*theta_))*(y/factor) + np.cos(i*theta_)*(x/factor))-r
        f = f*fi
    f = np.power(-1, N)*f
    return f-c

In [6]:
def r_solution_initial(r0, theta, c, m):
    data = (theta, c, m)
    r,_,ier,mesg = fsolve(r2fun, r0, args=data, full_output=True)
    r_value = r.item(0)
    if ier != 1:
        print ("r_solution:I coudn't solve the algebraic constraint, error:\n\n:",r_value, theta,c, m)
    sys.stdout.flush()
    #r_value = sol.x
    return r_value

In [7]:
def r_solution_array(r0, theta, c, m):
    data = (theta, c, m)
    r,_,ier,mesg = fsolve(r2fun, r0, args=data, full_output=True)
    if ier != 1:
        print ("r_solution:I coudn't solve the algebraic constraint, error:\n\n:",c, m)
    sys.stdout.flush()
    return r

In [8]:
def Radius2fun(dimension,theta_r, c_r, m):
    global N, r
    r0 = r
    radius = np.zeros(shape = (dimension, dimension))
    for i in range(dimension):
        c = c_r[i]
        if i <=1:
            for j in range(dimension):
                theta = theta_r[j]
                R = r_solution_initial(r0, theta, c, m)
                radius[i][j] = R
                r0 = R
        else:
            r0 = radius[i-1]
            R = r_solution_array(r0, theta_r, c, m)
            radius[i] = R
    return radius

In [9]:
def dev_r_theta2fun(theta_ranges, Radius, c_ranges, m):
    global r, c_max
    C_N_factor = np.reshape(((1 + c_ranges/c_max)**m), (-1,1))
    Sin = np.sin(theta_ranges)
    Cos = np.cos(theta_ranges)
    R_sin = Radius*Sin
    R_cos = Radius*Cos
    eq1 = a1*R_sin+b1*R_cos-r*C_N_factor
    eq2 = a2*R_sin+b2*R_cos-r*C_N_factor
    eq3 = a3*R_sin+b3*R_cos-r*C_N_factor
    eq4 = a4*R_sin+b4*R_cos-r*C_N_factor
    eq5 = a5*R_sin+b5*R_cos-r*C_N_factor
    eq6 = a6*R_sin+b6*R_cos-r*C_N_factor
############################
    d_eq1_1 = a1*Sin+b1*Cos
    d_eq2_1 = a2*Sin+b2*Cos
    d_eq3_1 = a3*Sin+b3*Cos
    
    d_eq4_1 = a4*Sin+b4*Cos
    d_eq5_1 = a5*Sin+b5*Cos
    d_eq6_1 = a6*Sin+b6*Cos
###########################
    d_eq1_2 = a1*R_cos-b1*R_sin
    d_eq2_2 = a2*R_cos-b2*R_sin
    d_eq3_2 = a3*R_cos-b3*R_sin
    d_eq4_2 = a4*R_cos-b4*R_sin
    d_eq5_2 = a5*R_cos-b5*R_sin
    d_eq6_2 = a6*R_cos-b6*R_sin
###
    z_denominator = (d_eq1_1*eq2*eq3*eq4*eq5*eq6+d_eq2_1*eq1*eq3*eq4*eq5*eq6+d_eq3_1
                     *eq2*eq1*eq4*eq5*eq6+d_eq4_1*eq2*eq3*eq1*eq5*eq6+
                     d_eq5_1*eq2*eq3*eq4*eq1*eq6+d_eq6_1*eq2*eq3*eq4*eq5*eq1)
    z_numerator = -(d_eq1_2*eq2*eq3*eq4*eq5*eq6+d_eq2_2*eq1*eq3*eq4*eq5*eq6+d_eq3_2
                     *eq2*eq1*eq4*eq5*eq6+d_eq4_2*eq2*eq3*eq1*eq5*eq6+
                     d_eq5_2*eq2*eq3*eq4*eq1*eq6+d_eq6_2*eq2*eq3*eq4*eq5*eq1)
    dev_r_theta = z_numerator/z_denominator
    return dev_r_theta

In [10]:
def dev_r_C2fun(theta_ranges, Radius, c_ranges, m):
    global r, c_max, N
    c = np.reshape(c_ranges, (-1,1))
    C_N_factor = (1+(c/c_max))**m
    Sin = np.sin(theta_ranges)
    Cos = np.cos(theta_ranges)
    R_sin = Radius*Sin
    R_cos = Radius*Cos
    
    eq1 = a1*R_sin+b1*R_cos-r*C_N_factor
    eq2 = a2*R_sin+b2*R_cos-r*C_N_factor
    eq3 = a3*R_sin+b3*R_cos-r*C_N_factor
    eq4 = a4*R_sin+b4*R_cos-r*C_N_factor
    eq5 = a5*R_sin+b5*R_cos-r*C_N_factor
    eq6 = a6*R_sin+b6*R_cos-r*C_N_factor
    
    ############################    
    d_eq1_1 = a1*Sin+b1*Cos
    d_eq2_1 = a2*Sin+b2*Cos
    d_eq3_1 = a3*Sin+b3*Cos
    
    d_eq4_1 = a4*Sin+b4*Cos
    d_eq5_1 = a5*Sin+b5*Cos
    d_eq6_1 = a6*Sin+b6*Cos
    ###########################
    z_denominator = (d_eq1_1*eq2*eq3*eq4*eq5*eq6+d_eq2_1*eq1*eq3*eq4*eq5*eq6+d_eq3_1
                     *eq2*eq1*eq4*eq5*eq6+d_eq4_1*eq2*eq3*eq1*eq5*eq6+
                     d_eq5_1*eq2*eq3*eq4*eq1*eq6+d_eq6_1*eq2*eq3*eq4*eq5*eq1)
    c_n_factor = m*((1+c/c_max)**(m-1))*(1/c_max)
    C_derv = C_N_factor**N+c*N*(C_N_factor**(N-1))*c_n_factor
    z_numerator = C_derv+r*c_n_factor*(eq2*eq3*eq4*eq5*eq6+
                                       eq1*eq3*eq4*eq5*eq6+eq1*eq2*eq4*eq5*eq6
                                       +eq1*eq2*eq3*eq5*eq6+eq1*eq2*eq3*eq4*eq6
                                       +eq1*eq2*eq3*eq5*eq4)
    dev_r_c = z_numerator/z_denominator
    return dev_r_c

In [11]:
def dev_h_c2fun(c, n):
    return (n/np.power(c_max, n))*np.power(c,n-1)

In [12]:
def average_mainfun(v, n, m, Radius, theta_ranges, C_ranges):
    global c_max, r, N

    dimen_theta = len(theta_ranges)
    dimen_c = len(C_ranges)
    
    dev_h_c = dev_h_c2fun(C_ranges, n)
    ds_base = (1/2)*(Radius**2)*(2*(np.pi)/(dimen_theta*N))
    ds_2 = np.delete(ds_base, -1, axis=1)
    ds_3 = np.delete(ds_base, 0, axis=1)

    delta_base_2 = np.sum(ds_2, axis=1)
    delta_base_3 = np.sum(ds_3, axis=1)
    
    dela_base_average = 0.5*(delta_base_2+delta_base_3)
    

    
    
    dv = (dela_base_average*dev_h_c)*(c_max/dimen_c)
    dv2 = np.delete(dv, -1, axis=0)
    dv3 = np.delete(dv, 0, axis=0)
    dv_average = 0.5*(dv2+dv3)
    
    dv_sum = np.sum(dv_average)
    H = v/(dv_sum*N)
    
    
    
    dev_r_theta = dev_r_theta2fun(theta_ranges, Radius, C_ranges, m)
    dl = np.sqrt(Radius**2 + dev_r_theta**2)*(2*(np.pi)/(dimen_theta*N))
    

###################################################################################################    
    
    d_h = H*dev_h_c
    dh = d_h.reshape(-1,1)
    d_r_c = dev_r_C2fun(theta_ranges, Radius, C_ranges, m)
    
    #########################################################
    #d_r_c[0][dimen_c//2] = 0 #when c=0, the d_r_c is not derivative, so to avoid the singularity
    #########################################################
    
    
    
    ds_surface = np.sqrt(dh**2+d_r_c**2)*(c_max/dimen_c)
    
    

##################################################################################################    
    
    surface = dl*ds_surface 

    
    surface1 = np.delete(surface, -1, axis=0)
    surface2 = np.delete(surface1, -1, axis=1)
    
    surface3 = np.delete(surface, 0, axis=0)
    surface4 = np.delete(surface3, 0, axis=1)
    
    surface = 0.5*(surface2+surface4)
    
   
    
    
    surface_size = np.sum(surface)*N
    return H, surface_size

In [13]:
def Main_M(v, dimen, m_ranges, n_ranges):
    C_ranges = np.linspace(1e-16, 0.999*c_max, dimen)
    theta_ranges = np.linspace(0, (2*np.pi)/N, dimen)
    m_length = len(m_ranges)
    n_length = len(n_ranges)
    M_n_surface_matrix = np.zeros((m_length, n_length))
    M_n_height_matrix = np.zeros((m_length, n_length))
    for i in tqdm(range(m_length)):
        M_Radius = Radius2fun(dimen, theta_ranges, C_ranges, m_ranges[i])
        for j in range(n_length):
            n_h, n_s = average_mainfun(v, n_ranges[j],m_ranges[i], M_Radius, theta_ranges, C_ranges)
            M_n_surface_matrix[i][j] = n_s
            M_n_height_matrix[i][j] = n_h
    surface_small = M_n_surface_matrix.min()
    index = np.where(M_n_surface_matrix == np.min(M_n_surface_matrix))
    height = M_n_height_matrix[index]
    return surface_small, height, M_n_surface_matrix, m_ranges[index[0]], n_ranges[index[1]]

In [14]:
Volume = [0.3]#,0.06,0.1,0.14, 0.18, 0.5, 1, 1.5]
#m_ranges = np.linspace(0.5, 1.5, 200)
#n_ranges = np.linspace(0.8,2, 200)
#m_ranges = np.linspace(0.3, 0.7, 200)
#n_ranges = np.linspace(0.6,1, 200)
#m_ranges = np.linspace(1, 1.3, 200)
#n_ranges = np.linspace(0.8,1.2, 200)
#m_ranges = np.linspace(0.3, 0.7, 200)
#n_ranges = np.linspace(0.9,1.2, 200)
m_ranges = np.arange(-3, 3, 0.1)
n_ranges = np.arange(0.5,2, 0.01)

In [15]:
ss = []
hh = []
s_matrixs = []
m_indexs = []
n_indexs = []
Dimen = 200####recommand 200
for v in Volume:
    s, h, s_matrix, m, n= Main_M(v, Dimen, m_ranges, n_ranges)
    ss.append(s)
    hh.append(h)
    s_matrixs.append(s_matrix)
    m_indexs.append(m)
    n_indexs.append(n)

  0%|          | 0/60 [00:00<?, ?it/s]

In [16]:
m_indexs

[array([0.6])]

In [17]:
n_indexs

[array([0.93])]

In [18]:
hh

[array([0.469833])]

In [19]:
ss

[1.748296474443767]

In [20]:
surface_energy = s_matrixs[0]

In [21]:
savetxt('hex03.csv', surface_energy, delimiter=',')